In [1]:
# keras CNN predict air, between, ground or not jumping

In [2]:
!pip3 install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import pickle
from utils_misc import pickle_load_or_create

2024-06-23 18:35:36.744291: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-23 18:35:36.788579: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-23 18:35:37.514744: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
from DataGeneratorBordersDB import DataGeneratorSkillBorders

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
video_border_labels_path = 'df_video_border_labels_seq'
video_folder = '../videos/'

video_names = [
    '20240201_atelier_001.mp4',
    '20240201_atelier_002.mp4',
    '20240201_atelier_003.mp4',
    '20240201_atelier_004.mp4',
    '20240201_atelier_005.mp4',
    '20240209_atelier_006.mp4',
    '20240209_atelier_007.mp4',
    '20240209_atelier_008.mp4',
]

train_videos = [ video_folder + trainvid for trainvid in video_names]

In [6]:
config = pickle_load_or_create('convLSTM_20_3x3_rgb', lambda:{
    'convolution': (3,3),
    'time_length':16,
    'dim':64,
    'rgb':True
}, config=True)
df_labels = pickle_load_or_create(video_border_labels_path, lambda: [], config=False)
df_labels

,path,frame,border
0,20240201_atelier_001.mp4,0,9
1,20240201_atelier_001.mp4,1,9
2,20240201_atelier_001.mp4,2,9
3,20240201_atelier_001.mp4,3,9
4,20240201_atelier_001.mp4,4,9
...,...,...,...
2755,20240201_atelier_003.mp4,439,0
2756,20240201_atelier_003.mp4,440,1
2757,20240201_atelier_003.mp4,441,2
2758,20240201_atelier_003.mp4,442,2


In [7]:
df_labels.loc[df_labels.border == 5, 'border'] = 3
df_labels.loc[df_labels.border == 9, 'border'] = 4

In [8]:
def get_random_frame(videos, grey=True, scale=1):
    """
    videos: array of video_paths
    df_video_border_labels: panda dataframe ['path', 'frame', 'borderlabel']
          0 : ground
          1 : heels of ground
          2 : air
          3 : Fault
          4 : no skipper or not jumping
    """
    path = videos[np.random.randint(0, len(videos)-1)]
    
    cap = cv2.VideoCapture(path)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_nr = np.random.randint(0, video_length-1)
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nr)
    res, frame = cap.read()
    frame = cv2.resize(frame, dsize=(0,0), fx=scale, fy=scale)
    # frame = cv2.cvtColor(frame, 7)
    cap.release()
    # cv2.destroyAllWindows()

    return path, frame_nr, frame

In [9]:
unique_labels = df_labels['border'].unique()
unique_labels

array([4, 0, 1, 2, 8])

In [10]:
input_shape = get_random_frame(train_videos, grey=True, scale=0.4)[2].shape
input_shape

(768, 432, 3)

In [11]:
config

{'convolution': (3, 3), 'time_length': 16, 'dim': 64, 'rgb': True}

In [12]:
convolution = config['convolution']

In [13]:
convolution

(3, 3)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, TimeDistributed, Dense, Flatten
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
                     input_shape=(20, 64, 64, 3),
                     padding='same', return_sequences=True))
model.add(BatchNormalization())

model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
                     padding='same', return_sequences=True))
model.add(BatchNormalization())

# Uncomment if you want to add more ConvLSTM2D layers
# model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
#                      padding='same', return_sequences=True))
# model.add(BatchNormalization())

# Apply TimeDistributed Dense layer to each time step
model.add(TimeDistributed(Flatten()))  # Flatten each frame
model.add(TimeDistributed(Dense(128, activation='relu')))
model.add(TimeDistributed(Dense(len(unique_labels), activation='softmax')))

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/home/miked/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d (ConvLSTM2D)        │ (None, 20, 64, 64, 64) │       154,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 20, 64, 64, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)      │ (None, 20, 64, 64, 64) │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 20, 64, 64, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 20, 262144)     │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 20, 128)        │    33,554,560 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 20, 5)          │           645 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,005,509 (129.72 MB)

 Trainable params: 34,005,253 (129.72 MB)

 Non-trainable params: 256 (1.00 KB)

In [16]:
# Parameters
params = {'dim': (64,64),
          'time_length':16,
          'batch_size': 20,
          'n_classes': len(unique_labels),
          'n_channels': 3,
          'shuffle': True,
}

training_generator = DataGeneratorSkillBorders(train=True, **params)
test_generator = DataGeneratorSkillBorders(train=False, **params)

connection established
DataGeneratorSkillBorders init done
connection established
DataGeneratorSkillBorders init done


In [17]:
training_generator.batch_order

,videoID,batches_in_video,random_order,total_batches
0,5,20,1,20
1,1,24,2,44
2,2,38,3,82
3,3,23,4,105
4,4,19,5,124
5,6,16,6,140


In [18]:
%%time
X, y = training_generator.__getitem__(25)

vid_id_idx 1 vid_batch_nr 5
CPU times: user 1.93 s, sys: 75.1 ms, total: 2.01 s
Wall time: 439 ms


In [19]:
X.shape

(1, 20, 64, 64, 3)

In [20]:
X

array([[[[[0.2627451 , 0.28235294, 0.28627451],
          [0.26666667, 0.28627451, 0.28627451],
          [0.29019608, 0.32156863, 0.31764706],
          ...,
          [0.65098039, 0.6627451 , 0.6627451 ],
          [0.60784314, 0.61960784, 0.61960784],
          [0.3372549 , 0.34901961, 0.34901961]],

         [[0.24313725, 0.25882353, 0.25882353],
          [0.26666667, 0.28235294, 0.29019608],
          [0.24705882, 0.25882353, 0.25882353],
          ...,
          [0.61960784, 0.61960784, 0.61960784],
          [0.58039216, 0.58039216, 0.58039216],
          [0.57254902, 0.57254902, 0.56470588]],

         [[0.44705882, 0.48235294, 0.48627451],
          [0.44313725, 0.47843137, 0.48235294],
          [0.23529412, 0.25490196, 0.25490196],
          ...,
          [0.52156863, 0.52156863, 0.51372549],
          [0.52941176, 0.53333333, 0.51372549],
          [0.52156863, 0.5254902 , 0.50588235]],

         ...,

         [[0.33333333, 0.32156863, 0.29803922],
          [0.31764706,

In [21]:
y.shape

(1, 20, 1)

In [22]:
y

array([[[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [0],
        [0],
        [0],
        [0]]])

In [23]:
y.shape


(1, 20, 1)

In [24]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(training_generator, epochs=2, 
                    validation_data=test_generator, shuffle=False)

len called len is: 140
vid_id_idx 0 vid_batch_nr 0
vid_id_idx 0 vid_batch_nr 1
Epoch 1/2
vid_id_idx 0 vid_batch_nr 0
vid_id_idx 0 vid_batch_nr 1
  1/140 ━━━━━━━━━━━━━━━━━━━━ 21:13 9s/step - accuracy: 0.0000e+00 - loss: 2.0798vid_id_idx 0 vid_batch_nr 2
  2/140 ━━━━━━━━━━━━━━━━━━━━ 6:53 3s/step - accuracy: 0.2500 - loss: 1.5598     vid_id_idx 0 vid_batch_nr 3
  3/140 ━━━━━━━━━━━━━━━━━━━━ 6:48 3s/step - accuracy: 0.3889 - loss: 1.2710vid_id_idx 0 vid_batch_nr 4
  4/140 ━━━━━━━━━━━━━━━━━━━━ 6:42 3s/step - accuracy: 0.4792 - loss: 1.0832vid_id_idx 0 vid_batch_nr 5
  5/140 ━━━━━━━━━━━━━━━━━━━━ 6:45 3s/step - accuracy: 0.5433 - loss: 0.9498vid_id_idx 0 vid_batch_nr 6
  6/140 ━━━━━━━━━━━━━━━━━━━━ 6:40 3s/step - accuracy: 0.5708 - loss: 1.1850vid_id_idx 0 vid_batch_nr 7
  7/140 ━━━━━━━━━━━━━━━━━━━━ 6:39 3s/step - accuracy: 0.5760 - loss: 1.6338vid_id_idx 0 vid_batch_nr 8
  8/140 ━━━━━━━━━━━━━━━━━━━━ 6:39 3s/step - accuracy: 0.5704 - loss: 2.1547vid_id_idx 0 vid_batch_nr 9
  9/140 ━━━━━━━━━━━━━

2024-06-23 18:36:47.834228: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (1, 9, 64, 64, 3) where an element of shape (None, 20, 64, 64, 3) was expected.
Traceback (most recent call last):

  File "/home/miked/.local/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/home/miked/.local/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/home/miked/.local/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (1, 9, 64, 64, 3) where an element of shape (None, 20, 64, 64, 3) was expected.




 19/140 ━━━━━━━━━━━━━━━━━━━━ 6:13 3s/step - accuracy: 0.4258 - loss: 6.6336vid_id_idx 1 vid_batch_nr 0


2024-06-23 18:36:50.180531: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (1, 9, 64, 64, 3) where an element of shape (None, 20, 64, 64, 3) was expected.
Traceback (most recent call last):

  File "/home/miked/.local/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/home/miked/.local/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/home/miked/.local/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (1, 9, 64, 64, 3) where an element of shape (None, 20, 64, 64, 3) was expected.


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]]


InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: `generator` yielded an element of shape (1, 9, 64, 64, 3) where an element of shape (None, 20, 64, 64, 3) was expected.
Traceback (most recent call last):

  File "/home/miked/.local/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/home/miked/.local/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/home/miked/.local/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (1, 9, 64, 64, 3) where an element of shape (None, 20, 64, 64, 3) was expected.


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]] [Op:__inference_one_step_on_iterator_8762]

In [ ]:
pd.DataFrame(history.history)

In [ ]:
model.fit?

In [ ]:
with open('../models/last_cnn_model_history.pkl', 'wb') as handle:
    pickle.dump(history, handle)